# Bedrock AgentCore Identity를 사용한 보안 자격 증명 관리

이 워크샵은 AI 에이전트를 구축할 때 외부 서비스의 API 키와 자격 증명을 안전하게 관리하기 위해 Strands Agents를 Amazon Bedrock AgentCore Identity와 통합하는 방법을 보여줍니다.

## 개요

이 실습에서는 다음을 수행합니다:
- 보안 자격 증명 관리 문제에 대해 학습
- Bedrock AgentCore Identity 기능 이해
- 외부 서비스용 API 키 자격 증명 공급자 생성
- AI 에이전트에서 보안 자격 증명 검색 테스트
- 자격 증명 관리 모범 사례 탐색

## 사전 요구사항

이 실습을 시작하기 전에 다음이 준비되어 있는지 확인하세요:
- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID
- 테스트용 외부 API 키 (예: Exa API 키)

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Strands Agents와 Bedrock AgentCore Python SDK에 필요한 패키지를 설치하세요:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore rich

AWS 리전에 따라 Nova Pro 모델 ID를 설정하세요:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

## 원격 Exa MCP 연결을 위한 Exa API 키 가져오기

이 실습에서는 [원격 Exa MCP](https://docs.exa.ai/reference/exa-mcp)와 상호작용하여 Exa Search API를 통해 실시간 웹 검색을 수행할 것입니다. 

이를 위해서는 연결을 위한 Exa API 키가 필요합니다.

Exa MCP 서버 URL: ```https://mcp.exa.ai/mcp?exaApiKey=your-exa-api-key```

Exa API 키를 얻으려면 [Exa 로그인 페이지](https://dashboard.exa.ai/login)에서 이메일로 등록하세요.

그런 다음 Exa 대시보드의 [API 키 섹션](https://dashboard.exa.ai/api-keys)으로 이동하여 API 키를 생성하세요. API 키를 복사하여 아래 코드의 `EXA_API_KEY`에 붙여넣으세요...

## 자격 증명 관리 문제 이해

### 안전하지 않은 API 키 사용 시연

먼저 적절한 자격 증명 관리 없이 외부 서비스(Exa 검색)를 사용하려고 할 때 어떤 일이 발생하는지 시연해보겠습니다. 

이는 하드코딩되거나 유효하지 않은 API 키로 인해 발생하는 보안 위험과 인증 실패를 보여줍니다.

⚠️ **중요**: 플레이스홀더 API 키를 실제 Exa API 키로 교체하세요.

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# !-------- 여기에 EXA API 키를 업데이트하세요  --------!
EXA_API_KEY = "YOUR_EXA_API_KEY"

# MCP 서버에 연결
print("\nMCP 서버에 연결 중...")
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))

with exa_server:
    # 모든 도구 결합 - 모두 동일한 방식으로 작동합니다!
    mcp_tools = (
        exa_server.list_tools_sync()
    )

    print(f"사용 가능한 도구: {[tool.tool_name for tool in mcp_tools]}")
    
    # Exa MCP 도구로 에이전트 생성
    agent = Agent(model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
                  system_prompt="당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다.",
                  tools=mcp_tools)

    agent("Bedrock AgentCore란 무엇인가요?")

## Bedrock AgentCore Identity란 무엇인가요?

Amazon Bedrock AgentCore Identity는 외부 서비스에 액세스해야 하는 AI 에이전트를 위한 보안 자격 증명 관리를 제공합니다. 

주요 이점은 다음과 같습니다:

- **보안 저장소**: AWS Secrets Manager에 API 키, 토큰 및 자격 증명 저장
- **런타임 검색**: 하드코딩 없이 런타임에 보안 자격 증명 액세스 제공
- **액세스 제어**: 세밀한 액세스 권한을 위해 AWS IAM과 통합
- **감사 추적**: 보안 모니터링을 위한 자격 증명 액세스 로그 유지
- **순환 지원**: 자동 자격 증명 순환 및 수명 주기 관리 활성화

이 서비스는 애플리케이션 코드에 민감한 자격 증명을 하드코딩할 필요를 없애 보안 위험을 줄이고 규정 준수를 개선합니다.

## 보안 자격 증명 관리 생성

### 1단계: API 키 자격 증명 공급자 생성

이제 Bedrock AgentCore Identity를 사용하여 API 키 자격 증명 공급자를 생성하겠습니다. 

이는 Exa API 키를 AWS Secrets Manager에 안전하게 저장하고 코드에서 자격 증명을 노출하지 않고 액세스할 수 있는 관리형 방법을 제공합니다.

⚠️ **중요**: 플레이스홀더 API 키를 실제 Exa API 키로 교체하세요.

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
from botocore.exceptions import ClientError
import boto3

# !-------- 여기에 EXA API 키를 업데이트하세요  --------!
EXA_API_KEY = "YOUR_EXA_API_KEY"

region = boto3.session.Session().region_name

# API 키 공급자 구성
identity_client = IdentityClient(region=region)

try:
    api_key_provider = identity_client.create_api_key_credential_provider({
        "name": "exa-apikey-provider",
        "apiKey": EXA_API_KEY # 외부 애플리케이션 공급업체(예: OpenAI)에서 얻은 API 키로 교체하세요
    })
    print("AgentCore Identity API 키 자격 증명 공급자가 생성되었습니다.")
    print(api_key_provider)
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        print("AgentCore Identity API 키 자격 증명 공급자가 이미 존재합니다.")
    else:
        print(f"오류: {e}")
except Exception as e:
    # API 키 공급자 생성 중 오류 표시
    print(f"오류: {e}")

### console 확인 : API 키 자격 증명 공급자 생성

[AWS Console](https://us-west-2.console.aws.amazon.com/bedrock-agentcore/identity?region=us-west-2) 에서 AgentCore Identity 가 생성된 것을 확인해봅니다. 

### 2단계: 보안 자격 증명 검색 테스트

이제 보안 자격 증명 검색을 사용하여 에이전트를 테스트해보겠습니다. 

`@requires_api_key` 데코레이터는 런타임에 자격 증명 공급자에서 API 키를 자동으로 검색하여 보안 모범 사례를 유지하면서 코드에 하드코딩된 비밀이 없도록 보장합니다.

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore.identity.auth import requires_api_key

@requires_api_key(provider_name="exa-apikey-provider")
def need_api_key(*, api_key: str):
    print(f'비동기 함수용 API 키 수신: {api_key}')
    return api_key

EXA_API_KEY = need_api_key()

# EXA MCP 서버에 연결
print("\nMCP 서버에 연결 중...")
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))

with exa_server:
    # 모든 도구 결합 - 모두 동일한 방식으로 작동합니다!
    mcp_tools = (
        exa_server.list_tools_sync()
    )

    print(f"사용 가능한 도구: {[tool.tool_name for tool in mcp_tools]}")
    
    # Exa MCP 도구로 에이전트 생성
    agent = Agent(model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
                  system_prompt="당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다.",
                  tools=mcp_tools)

    agent("Bedrock AgentCore란 무엇인가요?")

에이전트가 요청을 처리하고 응답을 생성하는 방법을 이해하기 위해 에이전트 루프의 상세한 실행 흐름을 살펴보겠습니다:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("에이전트 루프 세부사항")
console.rule()
console.print(f"루프 수: {agent.event_loop_metrics.cycle_count}")

table = Table(title="에이전트 메시지", show_lines=True)
table.add_column("역할", style="green")
table.add_column("텍스트", style="magenta")
table.add_column("도구 이름", style="cyan")
table.add_column("도구 입력", style="cyan")
table.add_column("도구 결과", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

### console 확인

[AWS Console](https://us-west-2.console.aws.amazon.com/bedrock-agentcore/identity/exa-apikey-provider) 에서 API Key 사용현황을 확인해봅니다.


## 리소스 정리 (선택사항)

배포된 리소스를 정리합니다:

In [ ]:
import boto3

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)

try:
    print("AgentCore Identity 삭제 중...")
    agentcore_control_client.delete_api_key_credential_provider(name="exa-apikey-provider")
    print("✓ AgentCore Identity 삭제가 시작되었습니다")
except Exception as e:
    print(f"❌ 정리 중 오류 발생: {e}")
    print("일부 리소스를 수동으로 정리해야 할 수 있습니다.")

## 결론

이 실습에서 성공적으로 다음을 수행했습니다:

- ✅ 에이전트 애플리케이션에서 하드코딩된 API 키의 보안 위험 식별
- ✅ Bedrock AgentCore Identity를 사용하여 보안 API 키 자격 증명 공급자 생성
- ✅ 외부 서비스 통합을 위한 보안 자격 증명 검색 구현
- ✅ 웹 검색 기능을 위한 Exa API로 보안 자격 증명 액세스 테스트

## AgentCore Identity의 주요 이점

- **보안 자격 증명 저장**: API 키와 비밀의 암호화된 저장
- **액세스 제어**: 자격 증명 액세스를 위한 세밀한 권한
- **감사 추적**: 자격 증명 사용 및 액세스의 완전한 로깅
- **통합 준비**: MCP 서버 및 에이전트 워크플로와의 원활한 통합
- **모범 사례**: 하드코딩된 자격 증명 및 보안 취약점 제거
